# WMTS Tests

In [1]:
from maap.maap import MAAP
from maap_test_helper import *
import requests
from bs4 import BeautifulSoup


In [5]:
# papermill parameters cell

stage = "dit"


In [3]:
stage = "production"  # todo: fix dit, as it currently fails, but production succeeds

maap_host=populate_maap_api_host(stage)


In [4]:

get_capabilities_response = requests.get(f'https://{maap_host}/api/wmts/GetCapabilities')
assert get_capabilities_response.status_code == 200

soup = BeautifulSoup(get_capabilities_response.text, 'xml')

assert {x['name'] for x in soup.find_all('ows:Operation')} == {
    "GetCapabilities", "GetTile"}

layers = soup.find_all('Layer')
assert layers

assert {x.find("ows:Title").string for x in layers} == {
    'AfriSAR_UAVSAR_Coreg_SLC', 'AFLVIS2'}

tile_matrix = "5" 
tile_col = "17"
tile_row = "16"

for layer in layers:
  get_tile_url = layer.find("ResourceURL")['template']
  populated_url = get_tile_url.replace(
      "{TileMatrix}", tile_matrix).replace("{TileCol}", tile_col).replace("{TileRow}", tile_row)
  response = requests.get(populated_url)
  assert response.status_code == 200


AssertionError: 